In [1]:
import numpy as np

In [2]:
def _swap_rows(A, k, l):
    A[[k, l]] = A[[l, k]]
def transform_to_row_echelon_form(A):
    A = A.astype(float).copy()
    m, n = A.shape
    
    pivot_i, pivot_j = 0, 0
    
    while pivot_i < m and pivot_j < n:
        i_max = np.abs(A[pivot_i:, pivot_j]).argmax() + pivot_i
        
        if A[i_max, pivot_j] == 0:
            pivot_j += 1
            continue
        
        _swap_rows(A, i_max, pivot_i)
        for i in range(pivot_i + 1, m):
            mulcoef = A[i, pivot_j]/A[pivot_i, pivot_j]
            A[i] = A[i] - mulcoef*A[pivot_i]
            
        pivot_i += 1
        pivot_j += 1
    return A

def backsubstituon(A_aug):
    m, n = A_aug.shape
    b = A_aug[:, -1]
    x = np.empty(m)
    
    for i in range(m-1, -1, -1):
        x[i] = b[i]/A_aug[i,i]
        
        for k in range(i-1, -1, -1):
            b[k] -= A_aug[k, i]*x[i]
    return x

In [5]:
transform_to_row_echelon_form(np.array([[1,1,1],[3,2,1],[2,1,2]]))

array([[3.        , 2.        , 1.        ],
       [0.        , 0.33333333, 0.66666667],
       [0.        , 0.        , 2.        ]])

In [6]:
np.linalg.solve(np.array([[1,1,1],[3,2,1],[2,1,2]]), [15, 28, 23])

array([3., 7., 5.])

In [5]:
def solve_system(A, b):
    A_aug = np.append(A, b.reshape(-1, 1), axis=1)
    U_aug = transform_to_row_echelon_form(A_aug)
    x = backsubstituon(U_aug)
    return x

In [6]:
A = np.array([[3,1],
              [1,2]])

b = np.array([9,8])

x = np.array([2, 3])
my_x = solve_system(A, b)
np_x = np.linalg.solve(A,b)

assert np.allclose(np.dot(A, x), b)
assert np.allclose(np.dot(A, my_x), b)
assert np.allclose(np.dot(A, np_x), b)

In [7]:
A = np.array([[1,2,1],
              [3,8,1],
              [0,4,1]])

b = np.array([2,12,2])

x = np.array([2, 1, -2])
my_x = solve_system(A, b)
np_x = np.linalg.solve(A,b)

assert np.allclose(np.dot(A, x), b)
assert np.allclose(np.dot(A, my_x), b)
assert np.allclose(np.dot(A, np_x), b)

In [8]:
A = np.array([[3,-2,1],
              [1,1,1],
              [3,-2,-1]])

b = np.array([7,2,3])

my_x = solve_system(A, b)
np_x = np.linalg.solve(A,b)

assert np.allclose(np.dot(A, my_x), b)
assert np.allclose(np.dot(A, np_x), b)
np_x

array([ 1., -1.,  2.])

In [11]:
np.random.seed(42)


for i in range(5, 100):
    A = np.random.rand(i,i)
    b = np.random.rand(i)
    
    np_x = np.linalg.solve(A, b)
    my_x = solve_system(A, b)
    
    assert np.allclose(my_x, np_x)
    assert np.allclose(np.dot(A, my_x), b)
    assert np.allclose(np.dot(A, np_x), b)

array([[0.73253767, 0.24097277, 0.35975186, ..., 0.61652013, 0.23319581,
        0.7017046 ],
       [0.05575146, 0.88647624, 0.33020239, ..., 0.39085718, 0.13165355,
        0.90240057],
       [0.26842582, 0.21412464, 0.86338238, ..., 0.08913126, 0.35675069,
        0.51416464],
       ...,
       [0.17192619, 0.65245646, 0.92953688, ..., 0.76799159, 0.97337455,
        0.35076785],
       [0.98016937, 0.16960558, 0.53781531, ..., 0.45556121, 0.10261114,
        0.60963445],
       [0.04315505, 0.72156904, 0.45065992, ..., 0.37953837, 0.93003076,
        0.5321445 ]])

# Transform to row-echelon form with constructing E that maps A -> U

In [102]:
A = np.array([[1, 2, 1],
              [3, 8, 1],
              [0, 4, 1]])

A

array([[1, 2, 1],
       [3, 8, 1],
       [0, 4, 1]])

In [48]:
def M_permutation(A, x, y):
    n, m = A.shape
    
    E = np.identity(n)
    x_row = np.zeros(m)
    x_row[y] = 1
    E[x] = x_row
    
    y_row = np.zeros(m)
    y_row[x] = 1
    E[y] = y_row
    
    return E

In [49]:
M_permutation(A, 0, 1)

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [51]:
def M_adder(A, from_i, to_i, mulcoef):
    n, m = A.shape
    
    E = np.identity(n)
    
    E[to_i, from_i] = mulcoef
    
    return E

In [52]:
A_sample = np.array([[1, 2], [3, 4]])
E = M_adder(A_sample, 1, 0, -1)

E @ A_sample

array([[-2., -2.],
       [ 3.,  4.]])

In [103]:
def transform_to_row_echelon_form(A):
    A = A.astype(float).copy()
    m, n = A.shape
    
    E_list = []
    
    pivot_i, pivot_j = 0, 0
    
    while pivot_i < m and pivot_j < n:
        i_max = np.abs(A[pivot_i:, pivot_j]).argmax() + pivot_i
        
        if A[i_max, pivot_j] == 0:
            pivot_j += 1
            continue
        
        _swap_rows(A, i_max, pivot_i)
        E_list.append(M_permutation(A, i_max, pivot_i))
        for i in range(pivot_i + 1, m):
            mulcoef = A[i, pivot_j]/A[pivot_i, pivot_j]
            A[i] = A[i] - mulcoef*A[pivot_i]
            
            E_list.append(M_adder(A, pivot_i, i, -mulcoef))
            
        pivot_i += 1
        pivot_j += 1
    return A, E_list

In [118]:
A = np.array([[1, 2, 1],
              [3, 8, 1],
              [0, 4, 1]])

A

array([[1, 2, 1],
       [3, 8, 1],
       [0, 4, 1]])

In [119]:
U, E_list = transform_to_row_echelon_form(A)
U

array([[3.        , 8.        , 1.        ],
       [0.        , 4.        , 1.        ],
       [0.        , 0.        , 0.83333333]])

In [120]:
A

array([[1, 2, 1],
       [3, 8, 1],
       [0, 4, 1]])

In [122]:
E = E_list[0]
for E_l in E_list[1:]:
    E = E_l @ E
E

array([[ 0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ],
       [ 1.        , -0.33333333,  0.16666667]])

In [123]:
E @ A

array([[3.        , 8.        , 1.        ],
       [0.        , 4.        , 1.        ],
       [0.        , 0.        , 0.83333333]])

In [108]:
E = E_list[1]
A = E @ A
A

array([[ 3.        ,  8.        ,  1.        ],
       [ 0.        , -0.66666667,  0.66666667],
       [ 0.        ,  4.        ,  1.        ]])

In [109]:
E = E_list[2]
A = E @ A
A

array([[ 3.        ,  8.        ,  1.        ],
       [ 0.        , -0.66666667,  0.66666667],
       [ 0.        ,  4.        ,  1.        ]])

In [110]:
E = E_list[3]
A = E @ A
A

array([[ 3.        ,  8.        ,  1.        ],
       [ 0.        ,  4.        ,  1.        ],
       [ 0.        , -0.66666667,  0.66666667]])

In [111]:
E = E_list[4]
A = E @ A
A

array([[3.        , 8.        , 1.        ],
       [0.        , 4.        , 1.        ],
       [0.        , 0.        , 0.83333333]])

In [112]:
E = E_list[5]
A = E @ A
A

array([[3.        , 8.        , 1.        ],
       [0.        , 4.        , 1.        ],
       [0.        , 0.        , 0.83333333]])